# Example of high-level API
Demo of the use of Wikidata

## Wikidata
Wikidata is either accessible by low-level API (such as for search) or by high-level APIs

In [1]:
import requests

In [2]:
def search(query, lang='en'):
    url = 'https://www.wikidata.org/w/api.php'
    par = {
        'action': 'wbsearchentities',
        'format': 'json',
        'search': query,
        'language': lang
    }
    response = requests.get(url=url, params=par)
    return response.json()

In [3]:
options = search('berlin')

In [4]:
for option in options['search']:
    print(option)

{'repository': '', 'id': 'Q64', 'concepturi': 'http://www.wikidata.org/entity/Q64', 'title': 'Q64', 'pageid': 190, 'url': '//www.wikidata.org/wiki/Q64', 'label': 'Berlin', 'description': 'capital and largest city of Germany', 'match': {'type': 'label', 'language': 'en', 'text': 'Berlin'}}
{'repository': '', 'id': 'Q821244', 'concepturi': 'http://www.wikidata.org/entity/Q821244', 'title': 'Q821244', 'pageid': 774087, 'url': '//www.wikidata.org/wiki/Q821244', 'label': 'Berlin', 'description': 'city in Coos County, New Hampshire, USA', 'match': {'type': 'label', 'language': 'en', 'text': 'Berlin'}}
{'repository': '', 'id': 'Q614184', 'concepturi': 'http://www.wikidata.org/entity/Q614184', 'title': 'Q614184', 'pageid': 578438, 'url': '//www.wikidata.org/wiki/Q614184', 'label': 'Berlin', 'description': 'town in Maryland, United States', 'match': {'type': 'label', 'language': 'en', 'text': 'Berlin'}}
{'repository': '', 'id': 'Q1569850', 'concepturi': 'http://www.wikidata.org/entity/Q1569850'

### High-level API

In [7]:
from wikidata.client import Client

In [8]:
client = Client()

In [9]:
entity = client.get('Q64', load=True)

In [10]:
dir(entity)

['__abstractmethods__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'attributes',
 'client',
 'data',
 'description',
 'get',
 'getlist',
 'id',
 'items',
 'iterlists',
 'iterlistvalues',
 'keys',
 'label',
 'lists',
 'listvalues',
 'load',
 'type',
 'values']

In [11]:
entity.attributes.keys()

dict_keys(['pageid', 'ns', 'title', 'lastrevid', 'modified', 'type', 'id', 'labels', 'descriptions', 'aliases', 'claims', 'sitelinks'])

In [12]:
for prop in entity.keys():
    values = entity.attributes['claims'][prop.id]
    for value in values:
        print(prop.id, value['mainsnak']['datavalue'], value['mainsnak']['datatype'])

P610 {'value': {'entity-type': 'item', 'numeric-id': 19259618, 'id': 'Q19259618'}, 'type': 'wikibase-entityid'} wikibase-item
P1151 {'value': {'entity-type': 'item', 'numeric-id': 3248436, 'id': 'Q3248436'}, 'type': 'wikibase-entityid'} wikibase-item
P31 {'value': {'entity-type': 'item', 'numeric-id': 1901835, 'id': 'Q1901835'}, 'type': 'wikibase-entityid'} wikibase-item
P31 {'value': {'entity-type': 'item', 'numeric-id': 200250, 'id': 'Q200250'}, 'type': 'wikibase-entityid'} wikibase-item
P31 {'value': {'entity-type': 'item', 'numeric-id': 257391, 'id': 'Q257391'}, 'type': 'wikibase-entityid'} wikibase-item
P31 {'value': {'entity-type': 'item', 'numeric-id': 1549591, 'id': 'Q1549591'}, 'type': 'wikibase-entityid'} wikibase-item
P31 {'value': {'entity-type': 'item', 'numeric-id': 1221156, 'id': 'Q1221156'}, 'type': 'wikibase-entityid'} wikibase-item
P31 {'value': {'entity-type': 'item', 'numeric-id': 1307779, 'id': 'Q1307779'}, 'type': 'wikibase-entityid'} wikibase-item
P31 {'value': {

In [13]:
def location(entity):
    lat, lon = None, None
    try:
        e = entity.attributes['claims']['P625'][0]['mainsnak']['datavalue']
        lat, lon = e['value']['latitude'], e['value']['longitude']
    except:
        pass
    return lat, lon

In [14]:
for option in options['search']:
    entity = client.get(option['id'], load=True)
    print(option['label'], location(entity))
    print(entity.description, '\n')

Berlin (52.516666666667, 13.383333333333)
capital and largest city of Germany 

Berlin (44.468611111111, -71.183888888889)
city in Coos County, New Hampshire, USA 

Berlin (38.3333, -75.2167)
town in Maryland, United States 

Berlin (43.969722222222, -88.948611111111)
city in Green Lake and Waushara Counties, Wisconsin, USA 

Berlin (39.7921, -74.937)
borough in Camden County, New Jersey, United States 

Berlin (41.613888888889, -72.7725)
town in Connecticut 

Berlin (None, None)
family name 



## SPARQL

In [15]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [16]:
endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT DISTINCT *
WHERE
{
  ?city wdt:P17 ?country.
  ?city wdt:P31 wd:Q515.
  ?country wdt:P31 wd:Q6256.
  ?city rdfs:label ?cityLabel.
  ?country rdfs:label ?countryLabel.
  FILTER (langMatches( lang(?cityLabel), "en" ) ).
  FILTER (langMatches( lang(?countryLabel), "en" ) ).
}
LIMIT 100
"""


def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)

{'country': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q16'}, 'city': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q23021'}, 'cityLabel': {'xml:lang': 'en-ca', 'type': 'literal', 'value': 'Fort St. John, British Columbia'}, 'countryLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Canada'}}
{'country': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q16'}, 'city': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q23021'}, 'cityLabel': {'xml:lang': 'en-gb', 'type': 'literal', 'value': 'Fort St. John'}, 'countryLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Canada'}}
{'country': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q16'}, 'city': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q23021'}, 'cityLabel': {'xml:lang': 'en-ca', 'type': 'literal', 'value': 'Fort St. John, British Columbia'}, 'countryLabel': {'xml:lang': 'en-ca', 'type': 'literal', 'value': 'Canada'}}
{'country': {'type': 'uri', 'value': 'http: